In [1]:
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
import pandas as pd

# Load Data into dataframes
co2_data = pd.read_csv('data/co2.csv', names=['Time','CO2'])
temperature_data = pd.read_csv('data/temp.csv', names=['Time','Temperature'])
humidity_data = pd.read_csv('data/humidity.csv', names=['Time','Humidity'])
# Create dictionary mapping option labels to their respective dataframes and units for graph generation
sensor_dict = {'CO2':co2_data, 'Temperature':temperature_data, 'Humidity':humidity_data}
sensor_unit_dict = {'CO2':'ppm', 'Temperature':'°F', 'Humidity':'%'}
    
# Set theme for plot
pio.templates.default = 'plotly_white'
'''
* Build Application for plot.
* Create HTML elements and items used to filter graph.
'''
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1('Mushroom Sensor Dashboard'),
    html.Label([
        'Select Sensor(s)',
        dcc.Dropdown(
            id='sensor-dropdown', multi=True,
            value='', options=[{'label': c, 'value': c} for c in ['CO2', 'Temperature', 'Humidity']])
    ]),
    dcc.Graph(id='sensor_graph'),
    html.Label([
    'Rolling Average Window Value',
        dcc.Slider(0, 250, value=120, id='window-slider',
        tooltip={"placement": "bottom", "always_visible": True})])
])

# Triggers upon user interaction with dcc items.
@app.callback(
    Output('sensor_graph', 'figure'),
    [Input('sensor-dropdown', 'value'),
     Input('window-slider', 'value')]
)
def update_figure(sensor, window):
    ''' 
    Function to update graph. Filters on selected sensors and window for rolling average             
            Parameters:
                    sensor (str/list): String if one item -> convert to list, list of strings if multiple items.
                    window (int): Integer used to calculate rolling average window.

            Returns:
                    fig (graph): Renders new graph based on selected criteria
    '''
    # Calculate rolling average window values
    co2_data['Rolling'] = co2_data['CO2'].rolling(window).mean()
    temperature_data['Rolling'] = temperature_data['Temperature'].rolling(window).mean()
    humidity_data['Rolling'] = humidity_data['Humidity'].rolling(window).mean()
    # Create figure with option for 2nd axis
    fig = make_subplots(specs=[[{'secondary_y': True}]])
    # Used to track second instance of data being plotted to utilize 2nd y-axis
    second_axis_bool = False
    ''' 
    Check if sensor is empty, a string, or a list. 
    * If it is empty, return an empty plot
    * If it is a string, convert to a list
    * If it is a list, generate the selected plots. 
        - Iterate over sensors and access their respective dataframe from the sensor_dict
    '''
    # Empty string, nothing to plot return empty plot
    if not sensor:
        return fig
    # Check if sensor is str or list, if it is a string then convert it to a list.
    if isinstance(sensor, str):
        sensor = [sensor]
    # Iterate over sensor list
    for item in sensor:
        fig.update_layout(title_text=' vs '.join(sensor))
        fig.add_trace(
            go.Scatter(x=sensor_dict[item]['Time'], y=sensor_dict[item][item],
            name=(str(item) + ' Data'),), secondary_y=second_axis_bool)
        fig.update_yaxes(ticksuffix=str(sensor_unit_dict[item]), title_text=item, secondary_y=second_axis_bool)
        fig.add_trace(
            go.Scatter(x=sensor_dict[item]['Time'], y=sensor_dict[item]['Rolling'],
            name=(str(item) + ' Rolling Average Data')), secondary_y=second_axis_bool)
        fig.update_xaxes(title_text='Date')
        second_axis_bool = True
    return fig

# Output will be within notebook
app.run_server(mode='inline')